In [1]:
import networkx as nx
import numpy as np
import sys

input source file: http://konect.cc/networks/adjnoun_adjacency/

In [2]:
g = nx.read_gml("adjnoun.gml")

In [3]:
node_dict = dict(zip(range(112),g.nodes))
key_list = list(node_dict.keys())
value_list = list(node_dict.values())
idx = key_list.index(0)
value_list.index("man")

1

In [4]:
g.number_of_nodes()

112

In [ ]:
g.number_of_edges()

# Finding Node-Adjacency Matrix

In [6]:
def get_node_adj_matrix_undirected(g):
    node_adj_matrix = np.zeros((g.number_of_nodes(),g.number_of_nodes()))
    zip_values = dict(zip(list(g.nodes),range(0,g.number_of_nodes())))
    for i,idx1 in list((zip(list(g.nodes),range(0,112)))):
        for idx2, j in enumerate(g.edges):
            if i==j[0]:
                node_adj_matrix[idx1][zip_values[j[1]]]=1
                node_adj_matrix[zip_values[j[1]]][idx1]=1
    return node_adj_matrix    

In [7]:
mat =(get_node_adj_matrix_undirected(g))

# Verifying if Graph is connected or not!

In [8]:
def is_graph_connected(g):
    visited = [False]*112
    def depth_first_search(g,v):
        visited[v]= True
        for i in g.neighbors(node_dict[v]):
            if visited[value_list.index(i)]==False:
                depth_first_search(g,value_list.index(i))
        return np.array(visited)
    if np.mean(depth_first_search(g,0)) == 1:
        return "It's a connected graph!"
    else:
        return "Not connected"

In [9]:
is_graph_connected(g)

"It's a connected graph!"

In [10]:
def median_degree_of_nodes(g):
    node_matrix = get_node_adj_matrix_undirected(g)
    in_deg = np.sum(node_matrix, axis = 1)
    sorted_deg = np.sort(np.array(in_deg))
    if len(g.nodes)%2 == 0:
        return float((sorted_deg[int(len(g.nodes)/2)]+sorted_deg[int((len(g.nodes)/2))-1])/2)
    else:
        return sorted_deg[int((len(g.nodes)-1)/2)]

In [11]:
median_degree_of_nodes(g)

6.0

# Finding number of Triangles with in the graph

In [12]:
def number_of_tri(g):
    node_matrix = np.matrix(get_node_adj_matrix_undirected(g))
#A^n[i][j] represents how many distinct paths are present with length of n originating from node i and reaching destination j
#Here, we are interested in number of triangles. So, length is 3 and since it's a closed polygon source and destination nodes are
# same
    return (np.trace(node_matrix*node_matrix*node_matrix)/6)
# We are dividing by 6, because same triangle can be formed from each node and since it's undirected graph two similar triangles
# say a-b-c and c-b-a are formed.

In [13]:
number_of_tri(g)

284.0

# Updating the Weights
    

In [14]:
node_values= dict((zip(list(g.nodes),range(0,112))))
key_list = list(node_values.keys())
value_list = list(node_values.values())
for i in list(g.edges):
    u,v = i
    g[u][v]["w"]=abs(value_list[key_list.index(u)]-value_list[key_list.index(v)])
edges=sorted(g.edges(data=True), key=lambda t: t[2].get('w', 1))
edges = np.array(edges)    

In [15]:
edges

array([['agreeable', 'man', {'w': 1}],
       ['man', 'old', {'w': 1}],
       ['old', 'person', {'w': 1}],
       ...,
       ['old', 'same', {'w': 102}],
       ['old', 'strong', {'w': 103}],
       ['old', 'year', {'w': 109}]], dtype=object)

# Kruskals Algorithm

In [42]:
def Kruskals(g):
    data_tuple = []
    edges_list=np.array(sorted(g.edges(data=True), key=lambda t: t[2].get('w', 1)))
    for i in edges_list:
        u,v,w = i
        data_tuple.append((value_list[key_list.index(u)], value_list[key_list.index(v)],g[u][v]["w"]))
    numVer = len(g.nodes)
    def find(main, i):
        if main[i] != i:
            main[i] = find(main, main[i])
        return main[i]
    def union(main, order, x, y):
        if order[x] < order[y]:
            main[x] = y
        elif order[x] > order[y]:
            main[y] = x
        else:
            main[y] = x
            order[x] += 1
    result = [] 
    i = 0
    edgeCount = 0
    data_tuple = sorted(data_tuple,key=lambda item: item[2])
    main = [i for i in range(len(g.nodes))]
    order = [0]*len(g.nodes)
    while edgeCount < (len(g.nodes)- 1):
        u, v, w = data_tuple[i]
        i = i + 1
        x = find(main, u)
        y = find(main, v)
        if x != y:
            edgeCount+=1
            result.append([u, v, w])
            union(main, order, x, y)
    minimumCost = 0
    for u, v, weight in result:
        minimumCost += weight
        print("Source-node :" + str(key_list[value_list.index(u)]) + " destination-node : " + str(key_list[value_list.index(v)]) + " cost :" + str(weight))
    print("Overall_cost", minimumCost)

In [43]:
Kruskals(g)

Source-node :agreeable destination-node : man cost :1
Source-node :man destination-node : old cost :1
Source-node :old destination-node : person cost :1
Source-node :anything destination-node : short cost :1
Source-node :short destination-node : arm cost :1
Source-node :arm destination-node : round cost :1
Source-node :aunt destination-node : first cost :1
Source-node :bad destination-node : air cost :1
Source-node :beautiful destination-node : black cost :1
Source-node :best destination-node : course cost :1
Source-node :better destination-node : heart cost :1
Source-node :place destination-node : right cost :1
Source-node :eye destination-node : bright cost :1
Source-node :bright destination-node : evening cost :1
Source-node :certain destination-node : day cost :1
Source-node :other destination-node : child cost :1
Source-node :child destination-node : happy cost :1
Source-node :dark destination-node : kind cost :1
Source-node :dear destination-node : good cost :1
Source-node :mothe

# Part d

In [65]:
def prim(g):
    visited = [False]*len(g.nodes)
    edgeNum = 0
    visited[0] = True
    mst = []
    node_values= dict((zip(list(g.nodes),range(0,112))))
    key_list = list(node_values.keys())
    value_list = list(node_values.values())
    edge_list = list(g.edges)
    def is_edge_present(u,v):
        if (u,v) in edge_list:
            return True
        else:
            return False
    def printmst(mst):
        print("Edge : Weight")
        for u,v,w in mst:
            print((u,v,w))
    while edgeNum< (len(g.nodes)-1):
        minimum = sys.maxsize
        for i in range(len(g.nodes)):
            if visited[i]== True:
                for j in range(len(g.nodes)):
                    if visited[j] == False and is_edge_present(key_list[value_list.index(i)],key_list[value_list.index(j)]) == True:
                        if minimum> g[key_list[value_list.index(i)]][key_list[value_list.index(j)]]["w"]:
                            minimum = g[key_list[value_list.index(i)]][key_list[value_list.index(j)]]["w"]
                            u = i
                            v = j
        mst.append((key_list[value_list.index(u)],key_list[value_list.index(v)], g[key_list[value_list.index(u)]][key_list[value_list.index(v)]]["w"]))
        visited[v] = True
        visited[u] = True
        edgeNum+=1
    printmst(mst)